In [8]:
# Importing the Libraries 
from helper import get_openai_api_key, get_llama_cloud_api_key
from IPython.display import display, HTML
from helper import extract_html_content
from llama_index.utils.workflow import draw_all_possible_flows
import os

# LlamaParse 
from llama_parse import LlamaParse

# For Building the Vector Index 
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex

# OpenAI 
from llama_index.llms.openai import OpenAI

# For Reading and Loading Vector Index 
from llama_index.core import StorageContext, load_index_from_storage

# For making RAG Agentic
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

In [9]:
# For Async Operations 
import nest_asyncio
nest_asyncio.apply()

In [6]:
# Getting the API Keys 
llama_cloud_api_key = get_llama_cloud_api_key()
openai_api_key = get_openai_api_key()

In [14]:
# For Parsing the Annual Report 
annual_report="./igl_annual_report.pdf"
base_url = "https://api.cloud.llamaindex.ai/api/parsing"
documents = LlamaParse(
    api_key=llama_cloud_api_key,
    # base_url=base_url,
    result_type="markdown",
    content_guideline_instruction="This is an corporate annual report, gather related facts together and format it as bullet points with header",
    verbose=True
).load_data(
    annual_report,
)

Started parsing the file under job_id 876fe88c-e26f-4538-a6de-361f80ae2a01
..

In [20]:
# print(documents)

In [24]:
# For Building the Vector Index 
index = VectorStoreIndex.from_documents(
    documents[:21],
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small", 
                                api_key= openai_api_key)
)

In [26]:
# Querying the Index 
llm = OpenAI(model="gpt-4o-mini")
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)
response = query_engine.query("What is the name of the company?")
print(response)

Indraprastha Gas Limited


In [25]:
# Storing the Index 
storage_dir = "./storage"
index.storage_context.persist(persist_dir=storage_dir)

In [27]:
# Reloading Index from Disk 
# Check if the index is stored on disk
if os.path.exists(storage_dir):
    # Load the index from disk
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    restored_index = load_index_from_storage(storage_context)
else:
    print("Index not found on disk.")

# Querying the Loaded Index 
response = restored_index.as_query_engine().query("What is the name of the company and its vision?")
print(response)

The name of the company is Indraprastha Gas. Its vision is focused on progress towards a sustainable future.


## Making RAG Agentic 

In [28]:
def query_report(q: str) -> str:
    """Answers questions about a specific annual report."""
    # we're using the query engine we already created above
    response = query_engine.query(f"This is a question about the given annual report : {q}")
    return response.response

In [29]:
annual_report_tool = FunctionTool.from_defaults(fn=query_report)

In [30]:
agent = FunctionCallingAgent.from_tools(
    tools=[annual_report_tool],
    llm=llm,
    verbose=True
)

In [31]:
response = agent.chat("What is the net profit of the company?")
print(response)

> Running step baa70463-d151-420c-9433-608ac2466b76. Step input: What is the net profit of the company?
Added user message to memory: What is the net profit of the company?
=== Calling Function ===
Calling function: query_report with args: {"q": "What is the net profit of the company?"}
=== Function Output ===
The net profit of the company for the year ended March 31, 2024, is Rs. 1,748.08 Crores.
> Running step 9a235a41-4617-41e3-8639-a91f52a98918. Step input: None
=== LLM Response ===
The net profit of the company for the year ended March 31, 2024, is Rs. 1,748.08 Crores.
The net profit of the company for the year ended March 31, 2024, is Rs. 1,748.08 Crores.
